In [ ]:
import pandas as pd
import numpy as np
import joblib
from pandarallel import pandarallel
import gc
import pandas as pd
import subprocess
import os
import threading
import time
import multiprocessing
import numpy as np
import random
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pty
import signal
import sys
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso,ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import gpytorch
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from sklearn.model_selection import KFold

from sklearn.model_selection import GridSearchCV
from scipy.stats import norm,expon
import scipy.stats as stats
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score, recall_score, precision_recall_curve, auc, roc_auc_score, roc_curve



from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer

# import pymc3 as pm

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim




import pickle

In [ ]:
windowsize = 10000
basepath='/benchmark/YCSB/result/cassandra/repeat/'
def zero_ratio(series):
    total_count = len(series)
    zero_count = (series == 0).sum()
    if total_count == 0:
        return 0
    else:
        return zero_count / total_count
    
def custom_percentiles4(series):
    percentile_columns = [0.25,0.5,0.75,0.95]
    if len(series) == 0:
        return pd.Series([-1] * len(percentile_columns), index=percentile_columns)
    else:
        return pd.Series(series.quantile(percentile_columns), index=percentile_columns)
    
def custom_percentiles_tail(series):
    percentile_columns = [0.25,0.5,0.75,0.9,0.95,0.99]
    if len(series) == 0:
        return pd.Series([0] * len(percentile_columns), index=percentile_columns)
    else:
        return pd.Series(series.quantile(percentile_columns), index=percentile_columns)
    
def custom_percentiles2(series):
    percentile_columns = [0.5, 0.95]
    if len(series) == 0:
        return pd.Series([-1] * len(percentile_columns), index=percentile_columns)
    else:
        return pd.Series(series.quantile(percentile_columns), index=percentile_columns)

def quantile_bins(group, num_bins):
    if len(group) < num_bins:
        return pd.Series(np.linspace(0, num_bins - 1, len(group)).astype(int))
    else:
        return pd.qcut(group, num_bins, labels=False, duplicates='drop')
    
def key_rangecout(x):
    return x.nunique()/x.count()

def filter_firstwindow(df):
    df['subgroup'] = (df.groupby('group').cumcount() // windowsize) 
    
    df['global_subgroup'] = df.groupby(['group', 'subgroup']).ngroup()

    df.reset_index(drop=True, inplace=True)
    # df = df.drop(columns=['group', 'subgroup'],axis=1)
    df = df.rename(columns={'group': 'group_raw'})
    df = df.rename(columns={'global_subgroup': 'group'})
    return df


def calculate_ape(y_true, y_pred):
    """ Calculate Absolute Percentage Error for each prediction """
    epsilon = 1e-7
    return np.abs((y_true - y_pred) / (y_true + epsilon))

def calculate_mape(row):
    actual = row[:-2]  # 最后两列是均值和方差，不包括在实际值中
    mean_value = row['mean']
    ape = calculate_ape(actual, mean_value)
    mape = ape.mean()
    return mape

#划分滑动窗口
def create_sliding_window(features, b_errors, window_size,X,y):
    for i in range(len(features) - window_size):
        X.append(features[i:(i + window_size)])
        y.append(b_errors[i + window_size])
    return X, y

window_size = 4

def getfeature(recordfile='record6_0.txt',distancefile='distance6_0.txt',filepath = '/benchmark/YCSB/result/cassandra/repeat/',type = 'full'):
    df = pd.DataFrame()
    separator_count = 0
    current_group_number = 0
    #分块读
    chunksize = 10000000
    #读取文件，使用逗号分隔
    
    
    for dftmp in pd.read_csv(filepath+recordfile, sep=',', header=None, names=['key', 'value','optype'], dtype={0: str, 1: np.float64,2:str},chunksize=chunksize):

        # 标记分隔符行
        dftmp['is_separator'] = dftmp['key'] == '**'

        separator_count += dftmp['is_separator'].sum()
        print(separator_count)
        # # 使用 cumsum 创建分组标签
        dftmp['group'] = dftmp['is_separator'].cumsum() + current_group_number
        
        # 更新当前group数量
        current_group_number = dftmp['group'].iloc[-1]

        # # 移除分隔符行
        dftmp = dftmp[~dftmp['is_separator']]

        # 转换第二列为数值类型（如果尚未转换）
        dftmp['value'] = pd.to_numeric(dftmp['value'], errors='coerce')
        dftmp['optype'] = pd.to_numeric(dftmp['optype'], errors='coerce')

        print(f"dftmp.shape:{dftmp.shape}")
        
        #分块的合并
        df = pd.concat([df,dftmp],ignore_index=True)


        
    # percentile_columns = ['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%']
    percentile_columns_tail =['25%', '50%', '75%', '90%', '95%', '99%']
    percentile_columns =['25%', '50%', '75%', '100%']
    
    #
    # with open(filepath + recordfile) as f:
    #     lines = f.readlines()
    # # 处理每一行并转为 DataFrame
    # data = [line.strip().split(',') for line in lines if line.strip()]  # 去除空行
    # df = pd.DataFrame(data, columns=['key', 'value', 'optype'])
    
    
    fe = pd.DataFrame()
    fe=pd.read_csv(filepath+distancefile, header=None,names=['Unique_Value_Sum'])
    df = df[:len(fe)]
    df['group'] = 0
    print(f"df length:{len(df)}-------distancelength:{len(fe)}")
    df['Unique_Value_Sum']=fe['Unique_Value_Sum'].values

    df = filter_firstwindow(df)
    all_groups = df['group'].unique()

    result_percentiles_sum = df[(df['optype'] == 0) & (df['value'] > 0)].groupby(['group'])['Unique_Value_Sum'].apply(custom_percentiles_tail)
    if result_percentiles_sum.empty:
        result_percentiles_sum = pd.DataFrame(0, index=all_groups, columns=percentile_columns_tail).reset_index()
        result_percentiles_sum.rename(columns={'index': 'group'}, inplace=True)
    else:
        result_percentiles_sum = result_percentiles_sum.unstack(level=-1).reset_index()
    percentile_columns_tmp = [i+'sum' for i in percentile_columns_tail]
    result_percentiles_sum.columns = ['group']+ percentile_columns_tmp

    result_nullpercentiles_sum = df[(df['optype'] == 0) & (df['value'] == 0)].groupby(['group'])['Unique_Value_Sum'].apply(custom_percentiles_tail)
    if result_nullpercentiles_sum.empty:
        result_nullpercentiles_sum = pd.DataFrame(0, index=all_groups, columns=percentile_columns_tail).reset_index()
        result_nullpercentiles_sum.rename(columns={'index': 'group'}, inplace=True)
    else:
        result_nullpercentiles_sum = result_nullpercentiles_sum.unstack(level=-1).reset_index()
    percentile_columns_tmp = [i+'null_sum' for i in percentile_columns_tail]
    result_nullpercentiles_sum.columns = ['group']+ percentile_columns_tmp

    get_num = df[(df['optype'] == 0) & (df['value'] !=0)].groupby('group')['value'].agg('count').reset_index()
    get_num.columns=['group','get_ct']
    getnull_num = df[(df['optype'] == 0) & (df['value'] ==0)].groupby('group')['value'].agg('count').reset_index()
    getnull_num.columns=['group','getnull_ct']
    set_num = df[df['optype'] == 1].groupby('group')['value'].agg('count').reset_index()
    set_num.columns=['group','set_ct']

    # 读写比
    result_zero_ratio = df.groupby('group')['optype'].agg('mean').reset_index()
    result_get_zero_ratio = df[df['optype'] == 0].groupby('group')['value'].agg([zero_ratio]).reset_index()
    get_key_range=df[(df['optype'] == 0) & (df['value'] > 0)].groupby('group')['key'].apply(key_rangecout).reset_index()


    # set value percentiles
    result_percentiles_set = df[df['optype'] == 1].groupby('group')['value'].apply(custom_percentiles4)
    if result_percentiles_set.empty:
        result_percentiles_set = pd.DataFrame(0, index=all_groups, columns=percentile_columns).reset_index()
        result_percentiles_set.rename(columns={'index': 'group'}, inplace=True)
    else:
        result_percentiles_set = result_percentiles_set.unstack(level=-1).reset_index()
    percentile_columns_tmp = [i+'set' for i in percentile_columns]
    result_percentiles_set.columns = ['group'] + percentile_columns_tmp

    # get value percentiles
    result_percentiles = df[(df['optype'] == 0) & (df['value'] > 0)].groupby('group')['value'].apply(custom_percentiles4)
    if result_percentiles.empty:
        result_percentiles = pd.DataFrame(0, index=all_groups, columns=percentile_columns).reset_index()
        result_percentiles.rename(columns={'index': 'group'}, inplace=True)
    else:
        result_percentiles = result_percentiles.unstack(level=-1).reset_index()
    percentile_columns_tmp = [i+'get' for i in percentile_columns]
    result_percentiles.columns = ['group'] + percentile_columns_tmp   

    if type == 'full':
        df['bins']=df[(df['optype'] == 0) & (df['value'] > 0)].groupby('group')['Unique_Value_Sum'].transform(quantile_bins, num_bins=4)
        result_percentiles_Unique_Value_Sum = df[(df['optype'] == 0) & (df['value'] > 0)].groupby(['group','bins'])['value'].apply(custom_percentiles4)
        tmp=pd.DataFrame(result_percentiles_Unique_Value_Sum.unstack(level=-1)).reset_index().drop('bins',axis=1)
        def funx(x):
            return pd.Series(x.iloc[:,1:].values.reshape(1,-1)[0])
        kkk=tmp.groupby('group').apply(lambda x:funx(x)).reset_index() 
        kkk=kkk.drop(0,axis=1)
        kkk.columns=['group']+[str(i) for i in kkk.columns[1:]]

    final_result = result_zero_ratio
    final_result = final_result.merge(get_num, on='group', how='left')
    final_result = final_result.merge(getnull_num, on='group', how='left')
    final_result = final_result.merge(set_num, on='group', how='left')

    final_result = final_result.merge(result_percentiles, on='group', how='left')
    if type == 'full':
        final_result = final_result.merge(kkk, on='group', how='left')
    final_result = final_result.merge(result_percentiles_set, on='group', how='left')
    final_result = final_result.merge(result_percentiles_sum, on='group', how='left')
    final_result = final_result.merge(result_nullpercentiles_sum, on='group', how='left')
    final_result = final_result.merge(get_key_range, on='group', how='left')

    final_result=final_result.fillna(0)
    return final_result
    # final_result.iloc[4999:5000,:]
    
    
def crps(y_true, y_pred_mean, y_pred_std):
    """
    parameters:
    y_true: 实际值
    y_pred_mean: 预测均值
    y_pred_std: 预测标准差
    
    return:
    crps_score
    """
    # 标准化实际值
    z = (y_true - y_pred_mean) / y_pred_std
    
    # 计算 CRPS
    crps_score = y_pred_std * (z * (2 * norm.cdf(z) - 1) + 2 * norm.pdf(z) - 1 / np.sqrt(np.pi))
    return crps_score


def fold_prediction(model,model_name):

    temp_train = X_train_or
    # x_train, y_train = temp_train[use_cols], temp_train[['label','std']]
    x_train, y_train = temp_train[use_cols], temp_train['label']     
    model.fit(x_train, y_train)
    with open(basepath+'model/'+model_name, 'wb') as file:
        pickle.dump(model, file)

def kfold_prediction(model,model_name):
    pred_test = np.zeros(len(X_test_or))
    pred_train = np.zeros(len(X_train_or))
    
    for index, (tr_idx, va_idx) in enumerate(kfold.split(all_index)):
        temp_train = X_train_or.iloc[tr_idx]
        temp_valid = X_train_or.iloc[va_idx]
        # x_train, y_train = temp_train[use_cols], temp_train[['label','std']]
        # x_valid, y_valid = temp_valid[use_cols], temp_valid[['label','std']]
        x_train, y_train = temp_train[use_cols], temp_train['label']
        x_valid, y_valid = temp_valid[use_cols], temp_valid['label']
        model.fit(x_train, y_train)
        pred_val = model.predict(x_valid)
        temp_valid['pre']=pred_val
        temp_valid['mape']=calculate_ape(y_valid,pred_val)

        pred_test+=model.predict(test_x)/foldnum
        pred_train[va_idx]=pred_val

        ape_values = calculate_ape(y_valid, pred_val)
        mape_test=np.sort(ape_values)[:].mean()
        ape_p95 = np.percentile(ape_values,95)
        ape_p99 = np.percentile(ape_values,99)

        print(f"{model_name} val:{index}, MAPE：{mape_test} p95_ape:{ape_p95} p99_ape:{ape_p99}")
    
    ape_values = calculate_ape(X_test_or['label'], pred_test)
    mape_test=np.sort(ape_values)[:].mean()
    percentile_95 = np.percentile(ape_values, 95)
    percentile_99 = np.percentile(ape_values, 99)
    print(f"{model_name} test: ,mean MAPE:{mape_test}, P95ape:{percentile_95}, P99ape:{percentile_99}")
    
    # 获取特征重要性
    feature_names = X_train_or[use_cols].columns
    if (model_name=='CatBoostRegressor'):
        feature_importances = model.get_feature_importance()
    elif(model_name in ['XGBRegressor','LGBMRegressor']):
        feature_importances = model.feature_importances_
    # 创建一个DataFrame来展示特征名和它们的重要性
    feature_importance_df = pd.DataFrame({'Feature Name': feature_names, 'Importance': feature_importances})
    # 按重要性降序排列
    feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    print(feature_importance_df)
    
all_filter_index = []
def noise_filter(row,noisetype ='b',alg = 'iqr',allindex = all_filter_index):
    if alg == 'iqr':
        #四分位点
        Q1 = np.percentile(row, 25)
        Q3 = np.percentile(row, 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # filtered_values = row[(row < upper_bound) &(row > lower_bound)]
        filtered_values = row[row < upper_bound]
        noise_b_list = row[row >= upper_bound]
    elif alg == 'sigma':
        #3sigma
        mean = np.mean(row)
        std = np.std(row)
        filtered_values = row[row < mean+3*std]
    elif alg == 'zscore':
        #z-score
        z_scores = zscore(row)
        filtered_values = row[np.abs(z_scores)<3]
    elif alg == 'lof':
        row_reshaped = row.values.reshape(-1, 1)
        lof = LocalOutlierFactor(n_neighbors=3)
        lof_fit = lof.fit_predict(row_reshaped)
        filtered_values = row[lof_fit == 1]
        noise_b_list = row[lof_fit == -1]
    elif alg == 'isof':
        # Isolation Forest
        row_reshaped = row.values.reshape(-1, 1)
        iso_forest = IsolationForest(contamination=0.2)
        iso_fit = iso_forest.fit_predict(row_reshaped)
        filtered_values = row[iso_fit != -1]
        noise_b_list = row[iso_fit == -1]
    elif alg == 'mad':
        # MAD (Median Absolute Deviation)
        median = np.median(row)
        mad = np.median([np.abs(x - median) for x in row])
        modified_z_scores = [0.6745 * (x - median) / mad for x in row]
        filtered_values = row[np.abs(modified_z_scores) < 3.5]
        noise_b_list = row[np.abs(modified_z_scores) >= 3.5]
    elif alg == 'dbscan':
        # DBSCAN
        row_reshaped = row.values.reshape(-1, 1)
        db = DBSCAN(eps=0.3, min_samples=3)
        db_fit = db.fit_predict(row_reshaped)
        filtered_values = row[db_fit != -1]
        noise_b_list = row[db_fit == -1]
    
    mean_val = np.mean(filtered_values)
    std_val = np.std(filtered_values)
    if noisetype=='a':
        filtered_values = filtered_values /mean_val
        all_filter_index.extend(filtered_values.tolist())
    elif noisetype=='b':
        # filtered_values = (filtered_values - filtered_values)/mean_val
        # all_filter_index.extend(filtered_values.tolist())
        # all_filter_index.extend(noise_b_list.tolist())
        noise_b_index = noise_b_list.index
        return noise_b_index
        
    return mean_val

class MAPEMetric(object):
    def get_final_error(self, error, weight):
        return np.mean(error)

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        # approxes 是预测值的列表，target 是真实值的列表
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])

        approx = approxes[0]
        error_sum = 0.0
        total_weight = 0.0

        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            error_sum += w * (np.abs(target[i] - approx[i]) / target[i])
            total_weight += w

        return error_sum / total_weight, total_weight
 

In [ ]:
final_result = getfeature(recordfile,distancefile,basepath)
labeltmp=pd.read_csv(basepath+'record'+clusterid+'.txtlable', sep=' ', header=None,names=['time'])
labeltmp = labeltmp.groupby(labeltmp.index // (windowsize/1000)).sum()
final_result['label'] = labeltmp['time']

In [ ]:
X_train_or, X_test_or, y_train_or, y_test_or=train_test_split(final_result, final_result, test_size=0.2, random_state=5)
X_train_or=X_train_or.reset_index(drop=True)
X_test_or=X_test_or.reset_index(drop=True)


# Kfold
error_data=[]
nerror_data=[]


foldnum=5
real_result_list=[]
all_index = range(len(X_train_or))
kfold = KFold(n_splits=foldnum, shuffle=True, random_state=4)
use_cols = [col for col in final_result.columns if col not in ['group','d','label','std','bin','time']]
label = 'time'  #一般是label
# '1','2','3','4','5','6','7','8','9','10','11','12','13','14','15'
def fold_prediction(model,model_name):

    temp_train = X_train_or
    # x_train, y_train = temp_train[use_cols], temp_train[['label','std']]
    x_train, y_train = temp_train[use_cols], temp_train[label]     
    model.fit(x_train, y_train)
    with open(basepath+'aliyunmodel/'+model_name, 'wb') as file:
        pickle.dump(model, file)

test_x = X_test_or[use_cols]
def kfold_prediction(model,model_name):
    pred_test = np.zeros(len(X_test_or))
    pred_train = np.zeros(len(X_train_or))
    
    for index, (tr_idx, va_idx) in enumerate(kfold.split(all_index)):
        temp_train = X_train_or.iloc[tr_idx]
        temp_valid = X_train_or.iloc[va_idx]
        # x_train, y_train = temp_train[use_cols], temp_train[['label','std']]
        # x_valid, y_valid = temp_valid[use_cols], temp_valid[['label','std']]
        x_train, y_train = temp_train[use_cols], temp_train[label]
        x_valid, y_valid = temp_valid[use_cols], temp_valid[label]
        model.fit(x_train, y_train)
        pred_val = model.predict(x_valid)
        temp_valid['pre']=pred_val
        temp_valid['mape']=calculate_ape(y_valid,pred_val)

        pred_test+=model.predict(test_x)/foldnum
        pred_train[va_idx]=pred_val

        ape_values = calculate_ape(y_valid, pred_val)
        mape_test=np.sort(ape_values)[:].mean()
        ape_p95 = np.percentile(ape_values,95)
        ape_p99 = np.percentile(ape_values,99)

        print(f"{model_name} val:{index}, MAPE：{mape_test} p95_ape:{ape_p95} p99_ape:{ape_p99}")
    
    ape_values = calculate_ape(X_test_or[label], pred_test)
    mape_test=np.sort(ape_values)[:].mean()
    percentile_95 = np.percentile(ape_values, 95)
    percentile_99 = np.percentile(ape_values, 99)
    print(f"{model_name} test: ,mean MAPE:{mape_test}, P95ape:{percentile_95}, P99ape:{percentile_99}")
    
    # 获取特征重要性
    feature_names = X_train_or[use_cols].columns
    if (model_name=='CatBoostRegressor'):
        feature_importances = model.get_feature_importance()
    elif(model_name in ['XGBRegressor','LGBMRegressor']):
        feature_importances = model.feature_importances_
    # 创建一个DataFrame来展示特征名和它们的重要性
    feature_importance_df = pd.DataFrame({'Feature Name': feature_names, 'Importance': feature_importances})
    # 按重要性降序排列
    feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    print(feature_importance_df)


def calculate_ape(y_true, y_pred):
    """ Calculate Absolute Percentage Error for each prediction """
    return np.abs((y_true - y_pred) / y_true)

def calculate_mape(row):
    actual = row[:-2]  # 最后两列是均值和方差，不包括在实际值中
    mean_value = row['mean']
    ape = calculate_ape(actual, mean_value)
    mape = ape.mean()
    return mape

In [ ]:
savemode = 1
model = LGBMRegressor(verbosity=-1,
                      metric='mse',
                      max_depth=8,
                      n_estimators=1000,
                      colsample_bytree=0.8,
                      lambda_l1=0.1,
                      objective='huber',
                      alpha=0.95
                      ) #metric：mse 
if (savemode):
    fold_prediction(model,"LGBM_base")
else:
    kfold_prediction(model,"LGBMRegressor")

# FT-Transformer

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred):
    epsilon = 1e-7 
    return torch.mean(torch.abs((y_true - y_pred) / (y_true + epsilon))) * 100


def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    # 确保cuDNN使用确定性算法
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False



df = final_result.copy()
use_cols = [col for col in final_result.columns if col not in ['group','label','std','bin','flag', 'time','normal','1', '2', '3', '4', '5', '6', '7', '8', '9', '10','11', '12', '13', '14', '15',]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 1. 准备数据
#

X = df[use_cols].values  # feature
y = df['label'].values  # label

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)  # 转换为列向量
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1).to(device)

empty_x_cat_train = torch.empty((X_train.shape[0], 0), dtype=torch.long).to(device)
empty_x_cat_test = torch.empty((X_test.shape[0], 0), dtype=torch.long).to(device)


class ContinuousTransformer(nn.Module):
    def __init__(self, input_dim, num_layers=6, nhead=8, dim_feedforward=64, dropout=0.1):
        super(ContinuousTransformer, self).__init__()
        # 将每个特征单独投影
        self.input_projection = nn.Linear(1, dim_feedforward)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=dim_feedforward,
            nhead=nhead,
            dropout=dropout,
            activation='gelu',
            batch_first=True  # 确保batch在第一个维度
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.output_layer = nn.Linear(dim_feedforward, 1)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x_cont):
        # x_cont形状: [batch_size, input_dim]
        # 为每个特征添加一个维度
        x_cont = x_cont.unsqueeze(2)  # 形状变为 [batch_size, input_dim, 1]
        x = self.input_projection(x_cont)  # 形状为 [batch_size, input_dim, dim_feedforward]
        x = self.dropout(x)
        x = self.transformer_encoder(x)  
        x = x.mean(dim=1)  # 聚合后形状为 [batch_size, dim_feedforward]
        output = self.output_layer(x)  # 输出形状为 [batch_size, 1]
        return output


# 获取输入特征的维度
input_dim = X_train.shape[1]

seed_everything(42)


# 实例化模型
model = ContinuousTransformer(input_dim=input_dim, num_layers=4, nhead=16, dim_feedforward=32, dropout= 0.0004058734643191623).to(device)


# 3. 定义损失函数和优化器
# criterion = nn.MSELoss()  # 均方误差损失函数
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=3.694663548346235e-05, weight_decay=1.6337705553617178e-05)

# 训练模型
num_epochs = 80 # 训练轮数
batch_size = 16  # 每个batch大小 32

train_data = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)


best_val_loss = float('inf')
patience = 5  # tolerant round

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        # outputs = model(inputs, x_cat=empty_x_cat_train)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        train_predictions = model(X_test)
        # train_predictions = model(X_test, x_cat=empty_x_cat_test)
        val_loss = criterion(train_predictions, y_test)
        train_mape = mean_absolute_percentage_error(y_test, train_predictions)
        
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0  
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("验证集性能不再提升，提前停止训练")
        break

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}-------{val_loss}, MAPE: {train_mape:.2f}%')

# 4. evaluattion model
model.eval() 
with torch.no_grad():
    predictions = model(X_test)
    # predictions = model(X_test,x_cat=empty_x_cat_test)
    test_loss = criterion(predictions, y_test)
    test_mape = mean_absolute_percentage_error(y_test, predictions)
    print(f'Test Loss: {test_loss.item():.4f}, Test MAPE: {test_mape:.2f}%')

# Uncertainty Model

In [ ]:

from gpytorch.kernels import RBFKernel, PeriodicKernel, SpectralMixtureKernel, ScaleKernel

# Setting the random seed
seed = 42
np.random.seed(seed)  
torch.manual_seed(seed)  
random.seed(seed)  

if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  
    

# feature_columns = ['optype', 'get_ct', 'getnull_ct', 'set_ct', '25%set', '50%set', '75%set', '100%set', 'key', 'time']
feature_columns = ['optype', 'get_ct', 'getnull_ct', 'set_ct', '25%get', '50%get',
       '75%get', '100%get', '25%set', '50%set', '75%set', '100%set',
       '25%sum', '50%sum', '75%sum', '90%sum', '95%sum', '99%sum','key', 'time']
# feature_columns = ['optype', 'get_ct', 'getnull_ct', 'set_ct', '25%get', '50%get',
#        '75%get', '100%get', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
#        '11', '12', '13', '14', '15', '25%set', '50%set', '75%set', '100%set',
#        '25%sum', '50%sum', '75%sum', '90%sum', '95%sum', '99%sum',
#        '25%null_sum', '50%null_sum', '75%null_sum', '90%null_sum',
#        '95%null_sum', '99%null_sum', 'key', 'time']
# feature_columns = [col for col in final_result.columns if col not in ['group', 'label', 'flag','ratio']]

# test CUDA 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')


scaler_X = StandardScaler()
final_result[feature_columns] = scaler_X.fit_transform(final_result[feature_columns])


percentile_99 = np.percentile(final_result['label'], 99.9)
filtered_labels = final_result[final_result['label'] <= percentile_99]['label']
scaler_y = StandardScaler()
scaler_y.fit(filtered_labels.values.reshape(-1, 1))
final_result['label'] = scaler_y.transform(final_result['label'].values.reshape(-1, 1)).flatten()

#  LSTM model
class LSTMFeatureExtractor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(LSTMFeatureExtractor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.ln = nn.LayerNorm(output_dim)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        final_output = lstm_out[:, -1, :]
        fc_output = self.fc(final_output)
        normalized_output = self.ln(fc_output)
        return normalized_output
    
input_dim = len(feature_columns)
lstm_extractor = LSTMFeatureExtractor(input_dim, hidden_dim=64, output_dim=64).to(device)


X = []
y = []
zero_indices = []
non_zero_indices = []
#slide window size
n = 10
sequence_length = 16000  


'''
使用LSTM-----------
'''
for start in range(0, len(final_result), sequence_length):
    sequence_data = final_result.iloc[start:start + sequence_length].copy()
    for i in range(len(sequence_data) - n):
        current_window = sequence_data.iloc[i:i + n][feature_columns].values
        current_window_tensor = torch.tensor(current_window, dtype=torch.float32).unsqueeze(0).to(device)
        lstm_feature = lstm_extractor(current_window_tensor)  
        # if(sequence_data.iloc[i + n - 1]['label']>5):
        #     continue
        X.append(lstm_feature.detach().cpu().numpy().flatten())  
        y.append(sequence_data.iloc[i + n - 1]['label'])
        
        if sequence_data.iloc[i + n-1]['flag'] == 0:
            zero_indices.append(len(y) - 1)  
        elif sequence_data.iloc[i + n-1]['flag'] == 1:
            non_zero_indices.append(len(y) - 1)


'''
concat
'''
# for start in range(0, len(final_result), sequence_length):
#     sequence_data = final_result.iloc[start:start + sequence_length].copy()
#     for i in range(len(sequence_data) - n):
#         # concat feature
#         current_window = sequence_data.iloc[i:i + n][feature_columns].values.flatten()  
#         X.append(current_window)
#         y.append(sequence_data.iloc[i + n - 1]['label'])  
        
#         if sequence_data.iloc[i + n - 1]['flag'] == 0:
#             zero_indices.append(len(y) - 1)  
#         elif sequence_data.iloc[i + n - 1]['flag'] == 1:
#             non_zero_indices.append(len(y) - 1)
            
'''
mean-pooling
'''
# for start in range(0, len(final_result), sequence_length):
#     sequence_data = final_result.iloc[start:start + sequence_length].copy()
#     for i in range(len(sequence_data) - n):
#         current_window = sequence_data.iloc[i:i + n][feature_columns].values  # (n, feature_dim)
#         mean_pooled_features = np.mean(current_window, axis=0) 
#         X.append(mean_pooled_features)
#         y.append(sequence_data.iloc[i + n - 1]['label'])  
        
#         if sequence_data.iloc[i + n - 1]['flag'] == 0:
#             zero_indices.append(len(y) - 1)  
#         elif sequence_data.iloc[i + n - 1]['flag'] == 1:
#             non_zero_indices.append(len(y) - 1)

'''
no encoder 
'''
# for start in range(0, len(final_result), sequence_length):
#     sequence_data = final_result.iloc[start:start + sequence_length].copy()
#     for i in range(len(sequence_data) - n):
#         current_window = sequence_data.iloc[i + n-1][feature_columns].values  # (n, feature_dim)
#         X.append(current_window)
#         y.append(sequence_data.iloc[i + n - 1]['label']) 
        
#         if sequence_data.iloc[i + n - 1]['flag'] == 0:
#             zero_indices.append(len(y) - 1)  
#         elif sequence_data.iloc[i + n - 1]['flag'] == 1:
#             non_zero_indices.append(len(y) - 1)



X = np.array(X)
y = np.array(y)

print(f"zero:{len(zero_indices)} nonzero:{len(non_zero_indices)}")

#下采样
if len(zero_indices) > len(non_zero_indices):
    downsampled_zero_indices = np.random.choice(zero_indices, size=len(zero_indices), replace=False)# kvrocks/redisaof 1/10 1/30 cassandra 1/10
    # downsampled_nonzero_indices = np.random.choice(non_zero_indices, size=len(non_zero_indices)//2, replace=False)
    balanced_indices = np.concatenate([downsampled_zero_indices, non_zero_indices])
    
else:
    balanced_indices = np.concatenate([zero_indices, non_zero_indices])
 
X_balanced = X[balanced_indices]
y_balanced = y[balanced_indices]


X_balanced = torch.tensor(X_balanced, dtype=torch.float32).to(device)
y_balanced = torch.tensor(y_balanced, dtype=torch.float32).to(device)



# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)


# 定义 GPyTorch 模型
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        input_dim = train_x.shape[-1]
        self.covar_module = gpytorch.kernels.ScaleKernel(
            RBFKernel(ard_num_dims=input_dim)
        )
        #PeriodicKernel(ard_num_dims=input_dim) + SpectralMixtureKernel(num_mixtures=4, ard_num_dims=input_dim)
        
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


# likelihood function
likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)#同方差假设
model = GPRegressionModel(X_train, y_train, likelihood).to(device)



# training model
model.train()
lstm_extractor.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': lstm_extractor.parameters()} 
], lr=0.005)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 100

print("Training the GPyTorch model...")
with tqdm(total=training_iterations) as pbar:
    for i in range(training_iterations):
        optimizer.zero_grad()
        
        output = model(X_train) #X_train
        
        loss = -mll(output, y_train)
        loss.backward()
        optimizer.step()
        pbar.set_postfix({'Loss': loss.item()})
        pbar.update(1)
    

# prediction
model.eval()
lstm_extractor.eval()
likelihood.eval()

print("Predicting on the training set...")
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    train_pred = likelihood(model(X_train))
    y_train_pred = train_pred.mean
    sigma_train = train_pred.stddev

print("Predicting on the test set...")
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    test_pred = likelihood(model(X_test))
    y_test_pred = test_pred.mean
    sigma_test = test_pred.stddev

y_train_pred = y_train_pred.cpu().numpy()
sigma_train = sigma_train.cpu().numpy()
y_test_pred = y_test_pred.cpu().numpy()
sigma_test = sigma_test.cpu().numpy()

y_train = y_train.cpu().numpy()
y_test = y_test.cpu().numpy()


y_train_pred_rescaled = scaler_y.inverse_transform(y_train_pred.reshape(-1, 1)).flatten()
y_test_pred_rescaled = scaler_y.inverse_transform(y_test_pred.reshape(-1, 1)).flatten()
sigma_train_rescaled = sigma_train * scaler_y.scale_
sigma_test_rescaled = sigma_test * scaler_y.scale_

y_train_rescaled = scaler_y.inverse_transform(y_train.reshape(-1, 1)).flatten()
y_test_rescaled = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()



# Visualisation of predicted results
plt.figure(figsize=(14, 7))

plt.subplot(2, 1, 1)
plt.scatter(range(len(y_train)), y_train_rescaled, label='Real Train')
plt.scatter(range(len(y_train)), y_train_pred_rescaled, label='Predicted Train', marker='x')
plt.fill_between(range(len(y_train)), y_train_pred_rescaled - 1.645 * sigma_train_rescaled, y_train_pred_rescaled + 1.645 * sigma_train_rescaled, alpha=0.2)
plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Train Data: Real vs Predicted')
plt.legend()

plt.subplot(2, 1, 2)
plt.scatter(range(len(y_test)), y_test_rescaled, label='Real Test')
plt.scatter(range(len(y_test)), y_test_pred_rescaled, label='Predicted Test', marker='x')
plt.fill_between(range(len(y_test)), y_test_pred_rescaled - 1.645 * sigma_test_rescaled, y_test_pred_rescaled + 1.645 * sigma_test_rescaled, alpha=0.2)
plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Test Data: Real vs Predicted')
plt.legend()
plt.ylim(0,20)

plt.tight_layout()
plt.show()